In [348]:
import numpy as np
import pandas as pd
import xgboost as xgb
import sklearn
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing.imputation import Imputer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score, roc_auc_score
from sklearn.cross_validation import StratifiedKFold
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
import os
import random
import matplotlib.pyplot as plt

In [349]:
def makeUserTarget(df, user):
    aux = []
    for _, row in df.iterrows():
        if row.our_data == 0:
            if(np.floor(row.SessionID/100).astype(int) == user):
                aux.append(1)
            else:
                aux.append(0)
        else:
            if 'Diego' in row.SessionID and user == 'Diego':
                aux.append(1)
            elif 'Toto' in row.SessionID and user == 'Toto':
                aux.append(1)
            elif (('Jo' in row.SessionID) or ('jo' in row.SessionID)) and user == 'Jo':
                aux.append(1)
            else:
                aux.append(0)
    df['target'] = aux

In [350]:
def separateIntoTrainAndTest(df, test_sessions = []):
    targetSessionIds = list(df[df.target == 1].SessionID.unique())
    targetChoosenSession = random.choice(targetSessionIds)
    notTargetSessionIds = list(df[df.target == 0].SessionID.unique())
    notTargetChoosenSessions = test_sessions
    for i in range(10):
        notTargetChoosenSession = random.choice(notTargetSessionIds) 
        notTargetChoosenSessions.append(notTargetChoosenSession)
        notTargetSessionIds.remove(notTargetChoosenSession)
    df_train = df[(df.SessionID != targetChoosenSession) & (~df.SessionID.isin(notTargetChoosenSessions))]
    df_test = df[(df.SessionID == targetChoosenSession) | (df.SessionID.isin(notTargetChoosenSessions))]
    
    return df_train, df_test

In [351]:
def getOurDataUsers(row):
    if 'Diego' in row.SessionID :
        return('Diego')
    elif 'Toto' in row.SessionID:
        return('Toto')
    elif ('Jo' in row.SessionID) or ('jo' in row.SessionID):
        return('Jo')

In [352]:
file_name = '../aggregated_features'
our_data_file_name = '../our_data_aggregated_features'
to_drop = []
df = pd.read_csv('./' + file_name + '.csv')
our_data_df = pd.read_csv('./' + our_data_file_name + '.csv')
df['our_data'] = 0
our_data_df['our_data'] = 1

In [353]:
users = (np.floor(df.SessionID/100).astype(int)).unique()
our_data_users = our_data_df.apply(getOurDataUsers,axis=1)

In [354]:
df = df.append(our_data_df,ignore_index=True)

In [ ]:
#user = random.choice(users)
user = 'Diego'
makeUserTarget(df, user)

In [ ]:
df = df.drop('our_data',axis=1)

In [ ]:
clf = xgb.XGBClassifier(n_estimators=90, max_depth=4, random_state=31, colsample_bytree=0.5, colsample_bylevel=0.5, learning_rate=0.2, subsample=0.9)
#clf = RandomForestClassifier()

In [ ]:
df_train, df_test = separateIntoTrainAndTest(df,test_sessions=['jo3','Toto2'])

In [ ]:
df_train = df_train.set_index(["SessionID", 'WindowNumber']).drop(to_drop, axis=1)
df_test = df_test.set_index(["SessionID", 'WindowNumber']).drop(to_drop, axis=1)

In [ ]:
x_train, y_train = df_train.drop("target", axis=1), df_train["target"]

In [ ]:
clf.fit(x_train, y_train)

In [ ]:
i_pred_proba = clf.predict_proba(x_train)

In [ ]:
auc = roc_auc_score(y_train, i_pred_proba[:, 1])
print(auc)

In [ ]:
x_test, y_test = df_test.drop("target", axis=1), df_test["target"]

In [ ]:
predict = clf.predict_proba(x_test)

In [ ]:
df_predict = pd.DataFrame(clf.predict_proba(x_test)[:, 1], columns=["prob"], index=x_test.index)

In [ ]:
df_socorro = df_predict.merge(df_test[['target']], right_index=True, left_index=True)

In [ ]:
df_socorro[df_socorro.target == 0].mean()

In [ ]:
df_socorro[df_socorro.target == 0].reset_index().groupby('SessionID').mean()

In [ ]:
df_socorro[df_socorro.target == 1].reset_index().groupby('SessionID').mean()

In [ ]:
xgb.plot_importance(clf)
plt.figure()